In [ ]:
%cd /Users/Deepthi/Documents/DSE/Capstone/Data/From Julius

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%cd ./../scraper/
%pwd

In [ ]:
%pwd

In [ ]:
womens_scrapped_cpp_br_pr = pd.read_csv('reviews_Women_all_final_scraped.csv.gz', compression='gzip')

In [ ]:
womens_scrapped_all = pd.read_hdf('womens_Meta_scraped.hdf5', key='data')

In [ ]:
import sklearn.feature_extraction.text 
from sklearn.feature_extraction.text  import TfidfVectorizer

In [ ]:
product_title = list(womens_scrapped_all['NAME'].drop_duplicates().dropna())

In [ ]:
my_stop_words = sklearn.feature_extraction.text.ENGLISH_STOP_WORDS.union({'women','womens'}) 

corpus = product_title

## Considered only 2-grams after trying 1-grams and 3-grams. 2-grams give more meaningful information
vectorizer = TfidfVectorizer(stop_words = set(my_stop_words),ngram_range=(2,2)) 

X = vectorizer.fit_transform(corpus)
idf = vectorizer.idf_

# print dict(zip(vectorizer.get_feature_names(), idf))


In [ ]:
# vectorizer.get_feature_names()

In [ ]:
z=vectorizer.inverse_transform(X)

In [ ]:
bi_grams = list(z[0])
for i in range(1,len(z)):
    bi_grams = bi_grams + list(z[i])

In [ ]:
# list(z[0]) + list(z[1])

In [ ]:
from collections import Counter
d= Counter(bi_grams)

df = pd.DataFrame.from_dict(d, orient='index').reset_index()

In [ ]:
df.columns = [['bigrams','count']]

In [ ]:
df.sort_values('count',ascending=False)

In [ ]:
len(df)

In [ ]:
len(df[df['count']>2])

In [ ]:
# df[df['count']>2]['bigrams']

In [ ]:
sel_bigrams=list(df[df['count']>2]['bigrams'])

In [ ]:
len(sel_bigrams)

In [ ]:
for i in range(len(sel_bigrams)):
    womens_scrapped_all[sel_bigrams[i]]=womens_scrapped_all['NAME'].fillna('0').str.lower()\
                                      .str.contains(sel_bigrams[i]).astype(int)

In [ ]:
# len(womens_scrapped_all.columns)

In [ ]:
# for i in range(1,10+1):
#     if i < 10:
#         print i
#     else:
#         print 'next number'

In [ ]:
## Create a feature vector field by concatenating each of the column generated using the 2-grams
womens_scrapped_all['feature_vector'] = womens_scrapped_all[sel_bigrams].transpose().apply(lambda x: np.array(x).tobytes())

# old version below
# womens_scrapped_all['feature_vector'] = "'"+womens_scrapped_all[sel_bigrams[0]].astype(str)

# for i in range(1,len(sel_bigrams)+1):
#     if i < len(sel_bigrams):
#         womens_scrapped_all['feature_vector'] = womens_scrapped_all['feature_vector'].astype(str)\
#                                             +womens_scrapped_all[sel_bigrams[i]].astype(str)
#     else:
#         womens_scrapped_all['feature_vector']=womens_scrapped_all['feature_vector']+"'"

# df_meta_only_womens['feature_vector']=df_meta_only_womens[0].str.cat(df_meta_only_womens[1])

In [ ]:
## Create a feature vector field by concatenating each of the column generated using the 2-grams

womens_scrapped_all['cnt'] = womens_scrapped_all[sel_bigrams[0]].astype(int)

for i in range(1,len(sel_bigrams)):
    womens_scrapped_all['cnt'] = womens_scrapped_all['cnt'].astype(int)\
                                        +womens_scrapped_all[sel_bigrams[i]].astype(int)

# df_meta_only_womens['feature_vector']=df_meta_only_womens[0].str.cat(df_meta_only_womens[1])

In [ ]:
# womens_scrapped_all['cnt']


In [ ]:
womens_scrapped_all.columns

In [ ]:
womens_scrapped_all[['asin','feature_vector']]

In [ ]:
# for i in range(len(womens_scrapped_all['feature_vector'])):
#     if (len(womens_scrapped_all['feature_vector'][i])) == 0:
#         print 1
# #     else:
# #         print 0

In [ ]:
# womens_scrapped_cpp_br_pr

In [ ]:
womens_scrapped_cpp_br_pr = womens_scrapped_cpp_br_pr.sort_values(by='asin')

In [ ]:
womens_scrapped_all = womens_scrapped_all.sort_values(by='asin')

In [ ]:
# womens_scrapped_cpp_br_pr_for_merge=womens_scrapped_cpp_br_pr[womens_scrapped_cpp_br_pr['asin'].isin(list(womens_scrapped_all['asin']))]

In [ ]:
# womens_scrapped_cpp_br_pr_for_merge

In [ ]:
final=womens_scrapped_cpp_br_pr.merge(womens_scrapped_all[['asin','feature_vector']], on='asin', how='left')\
[['reviewerID','asin','unixReviewTime','brand','price','feature_vector']]

In [ ]:
# final = pd.merge(womens_scrapped_cpp_br_pr,womens_scrapped_all[['asin','feature_vector']], on='asin',how='left')\
# [['reviewerId','asin','unixReviewTime','brand','price','feature_vector']]

In [ ]:
final = final.drop_duplicates()

In [ ]:
womens_scrapped_cpp_br_pr.count()

In [ ]:
# womens_scrapped_all.count()

In [ ]:
final.count()

In [ ]:
# 190695-183973

In [ ]:
womens_scrapped_cpp_br_pr.columns

In [ ]:
final.to_csv('reviews_Women_scraped_all_cpp_fv_2.csv',index=False)

In [ ]:
final.count()